<a href="https://colab.research.google.com/github/CelesteFernandezTomasincig/DataEngineering/blob/main/Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
import psycopg2
import yfinance as yf
import requests
from datetime import datetime, timedelta

In [82]:
#Extraccion data API
amzn = yf.Ticker('AMZN')

hist = amzn.history(period="1y")
hist['Date']=hist.index
hist=hist.reset_index(drop=True)
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
0,142.177994,145.937500,140.300003,144.596497,117316000,0.0,0.0,2022-04-28 00:00:00-04:00
1,129.848999,130.761002,121.625000,124.281502,272662000,0.0,0.0,2022-04-29 00:00:00-04:00
2,122.401001,124.667999,118.375000,124.500000,148788000,0.0,0.0,2022-05-02 00:00:00-04:00
3,124.053497,126.220497,122.824997,124.253502,79134000,0.0,0.0,2022-05-03 00:00:00-04:00
4,123.599998,126.000000,119.182999,125.928497,110746000,0.0,0.0,2022-05-04 00:00:00-04:00
...,...,...,...,...,...,...,...,...
246,106.099998,108.150002,105.080002,106.959999,86689700,0.0,0.0,2023-04-21 00:00:00-04:00
247,107.660004,109.230003,105.070000,106.209999,69575600,0.0,0.0,2023-04-24 00:00:00-04:00
248,104.910004,105.449997,102.449997,102.570000,65026800,0.0,0.0,2023-04-25 00:00:00-04:00
249,105.040001,106.620003,104.099998,104.980003,73803800,0.0,0.0,2023-04-26 00:00:00-04:00


In [83]:
#Limpieza de Data
hist= hist.rename(columns={'Open':'O', 'High':'H', 'Low':'L', 'Close':'C', 'Volume':'V','Dividens':'D', 'Stock Split':'S', 'Date':'Dat'})
hist= hist.drop(columns=['Dat'])
hist

,O,H,L,C,V,Dividends,Stock Splits
0,142.177994,145.937500,140.300003,144.596497,117316000,0.0,0.0
1,129.848999,130.761002,121.625000,124.281502,272662000,0.0,0.0
2,122.401001,124.667999,118.375000,124.500000,148788000,0.0,0.0
3,124.053497,126.220497,122.824997,124.253502,79134000,0.0,0.0
4,123.599998,126.000000,119.182999,125.928497,110746000,0.0,0.0
...,...,...,...,...,...,...,...
246,106.099998,108.150002,105.080002,106.959999,86689700,0.0,0.0
247,107.660004,109.230003,105.070000,106.209999,69575600,0.0,0.0
248,104.910004,105.449997,102.449997,102.570000,65026800,0.0,0.0
249,105.040001,106.620003,104.099998,104.980003,73803800,0.0,0.0


In [84]:
#Creo la tabla de destino en redshift 

In [85]:
#Establecer conexion con redshift 
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="celesteft00_coderhouse"
password='B2oKky9e5cJ7'

In [86]:
#Establecer conexion con redshift 
try: 
  conn = psycopg2.connect(
      host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
      dbname='data-engineer-database',
      user='celesteft00_coderhouse',
      password='B2oKky9e5cJ7',
      port='5439'
  )
  print ("Conexion exitosa")

except Exception as e:
    print("No se pudo conectar a Redshift")
    print(e)

Conexion exitosa


In [87]:
conn

<connection object at 0x7fd45df54f40; dsn: 'user=celesteft00_coderhouse password=xxx dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port=5439', closed: 0>

In [88]:
#Verificar que la tabla exista
cur = conn.cursor()
cur.execute("SELECT * FROM prices")
results = cur.fetchall()
results

[(Decimal('161.65'),
  Decimal('162.20'),
  Decimal('157.25'),
  Decimal('158.76'),
  79056000,
  Decimal('0.00'),
  Decimal('0.00')),
 (Decimal('158.40'),
  Decimal('160.08'),
  Decimal('154.51'),
  Decimal('157.78'),
  68136000,
  Decimal('0.00'),
  Decimal('0.00')),
 (Decimal('156.75'),
  Decimal('157.37'),
  Decimal('154.23'),
  Decimal('154.46'),
  46002000,
  Decimal('0.00'),
  Decimal('0.00')),
 (Decimal('152.71'),
  Decimal('154.14'),
  Decimal('150.53'),
  Decimal('151.12'),
  52112000,
  Decimal('0.00'),
  Decimal('0.00')),
 (Decimal('153.69'),
  Decimal('155.10'),
  Decimal('150.38'),
  Decimal('150.79'),
  55178000,
  Decimal('0.00'),
  Decimal('0.00')),
 (Decimal('150.02'),
  Decimal('156.02'),
  Decimal('149.60'),
  Decimal('155.54'),
  53390000,
  Decimal('0.00'),
  Decimal('0.00')),
 (Decimal('155.39'),
  Decimal('155.90'),
  Decimal('151.47'),
  Decimal('151.71'),
  51598000,
  Decimal('0.00'),
  Decimal('0.00')),
 (Decimal('151.52'),
  Decimal('154.04'),
  Decimal('15

In [89]:
#Enviar resultados
hist.head()

,O,H,L,C,V,Dividends,Stock Splits
0,142.177994,145.937500,140.300003,144.596497,117316000,0.0,0.0
1,129.848999,130.761002,121.625000,124.281502,272662000,0.0,0.0
2,122.401001,124.667999,118.375000,124.500000,148788000,0.0,0.0
3,124.053497,126.220497,122.824997,124.253502,79134000,0.0,0.0
4,123.599998,126.000000,119.182999,125.928497,110746000,0.0,0.0


In [96]:
from psycopg2.extras import execute_values
cur = conn.cursor()
table_name ="prices"
columns = ['O', 'H', 'L', 'C', 'V', 'D', 'S']
values = [tuple(x) for x in hist.to_numpy()]
insert_sql = f"INSERT INTO {table_name}({', '.join(columns)}) VALUES %s"
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute('COMMIT')

In [99]:
import os
print(os.getcwd())



/content


In [104]:
df = pd.read_csv("prices.csv")



FileNotFoundError: ignored

In [ ]:
 
# Eliminar los datos duplicados basados en todas las columnas
df_sin_duplicados = df.drop_duplicates()


In [ ]:
# Guardar la tabla sin duplicados en un archivo nuevo
df_sin_duplicados.to_csv("mi_tabla_sin_duplicados.csv", index=False)